
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

Le projet a été développé à l'aide de Alice Breton, étudiante à la maîtrise en génie informatique. Elle a suivi le cours lors de la session Hiver 2019.



# Projet : Débordement d'égouts

La description du projet est disponible à l'adresse suivante :
https://www.kaggle.com/t/a238b752c33a41d9803c2cdde6bfc929

Ce calepin Jupyter de base permet de charger et de nettoyer les données fournies. La dernière section détaille la génération du fichier des prédictions afin de le soumettre sur Kaggle dans le bon format.

Dans un premier temps, vous devrez récupérer l'archive *data.zip* sur Moodle. Ce dossier contient les fichiers suivants :
- surverses.csv
- precipitation.csv
- ouvrages-surverses.csv
- test.csv

Veuillez le décompresser dans le répertoire de ce calepin.

Le fichier *surverse.csv* répertorie s'il y a surverse (1) ou non (0) au cours de la journée pour les 170 ouvrages de débordement de 2013 à 2018 pour les mois de mai à octobre (inclusivement). Des renseignements additionnels sur les données sont disponibles à l'adresse suivante :

http://donnees.ville.montreal.qc.ca/dataset/debordement


Le fichier *precipitation.csv* contient les précipitations horaires en dixième de *mm* enregistrées à 5 stations pluviométriques de 2013 à 2019 :
- McTavish (7024745)
- Ste-Anne-de-Bellevue (702FHL8)
- Montreal/Pierre Elliott Trudeau Intl (702S006)
- Montreal/St-Hubert (7027329)
- L’Assomption (7014160)

Plus d'informations sur les précipitations sont disponibles à l'adresse suivante :

https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

Le fichier *ouvrages-surverses.csv* contient différentes caractéristiques des ouvrages de débordement. 

http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

Le fichier *test.csv* contient les ouvrages et les jours pour lesquels vous devez prédire s'il y a eu surverse (true) ou non (false). Notez que l'on s'intéresse ici à 5 ouvrages de débordement localisés tout autour de l'Ile de Montréal :
- 3260-01D dans Rivière-des-Prairies 
- 3350-07D dans Ahunstic 
- 4240-01D dans Pointe-aux-Trembles 
- 4350-01D dans le Vieux-Montréal 
- 4380-01D dans Verdun

#### Remarque

Dans le projet, on ne s'intéresse qu'aux surverses occasionnées par les précipitations. On ignore les surverses occasionnées par 
- fonte de neige (F)
- travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

On suppose que lorsqu'il n'y a pas de raison pour la surverse, il s'agit d'une surverse causée par les précipitations. Puisque Nous nous intéresserons uniquement aux surverses occasionnées par les précipitations liquides, nous ne considérons que les mois de mai à octobre inclusivement.

In [ ]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Distributions, Random, ScikitLearn

## Fonctions gobales

In [ ]:
"""
    splitdataframe(df::DataFrame, p::Real)

Partitionne en un ensemble d'entraînement et un ensemble de validation un DataFrame.

### Arguments
- `df::DataFrame` : Un DataFrame
- `p::Real` : La proportion (entre 0 et 1) de données dans l'ensemble d'entraînement.

### Détails

La fonction renvoie deux DataFrames, un pour l'ensemble d'entraînement et l'autre pour l'ensemble de validation.

### Exemple

\```
 julia> splitdataframe(df, p.7)
\```

"""
function splitdataframe(df::DataFrame, p::Real)
   @assert 0 <= p <= 1 
    
    n = size(df,1)
    
    ind = shuffle(1:n)
    
    threshold = Int64(round(n*p))
    
    indTrain = sort(ind[1:threshold])
    
    indTest = setdiff(1:n,indTrain)
    
    dfTrain = df[indTrain,:]
    dfTest = df[indTest,:]
    
    return dfTrain, dfTest
    
end

# Chargement des données et nettoyage préliminaire

## Chargement des surverses

In [ ]:
data = CSV.read("data/surverses.csv",missingstring="-99999")
first(data,5)

## Nettoyage des données sur les surverses

#### Extraction des surverses pour les mois de mai à octobre inclusivement

In [ ]:
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 
first(data,5)

#### Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"

In [ ]:
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison
first(data,5)

#### Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides

Ces facteurs correspondent à : 
- la fonte de neige (F), 
- les travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

In [ ]:
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data,5)

#### Exclusion des lignes où :SURVERSE est manquante

In [ ]:
surverse_df = dropmissing(data, disallowmissing=true)
first(surverse_df,5)

## Chargement des précipitations

In [ ]:
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)
first(data,5)

## Nettoyage des données sur les précipitations

#### Extraction des précipitations des mois de mai à octobre inclusivement

In [ ]:
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 
first(data,5)

 ### Remplissage des données manquantes
Nous allons tenter de remplir les données manquantes par des moyennes de précipitations lorsque les données sont inconnues pour 2 stations ou plus

In [ ]:
# fonction permettant de compter le nombre de valeurs du type "missing" dans une ligne
function countMissing(line)
    count=0
    ind = 0
    for i=1:length(line)
        if (ismissing(line[i]))
            count += 1
            ind = i
        end
    end
    return count, ind
end

In [ ]:
# fonction permettant de calculer le moyenne des valeurs d'une ligne peu importe le nombre de "missing"
function meanLine(line)
    values = []
    for i=1:length(line)
        if (!ismissing(line[i]))
            append!(values,line[i])
        end
    end
    return mean(values)
end

In [ ]:
# fonction permettant de remplacer toutes les valeurs "missing" d'une ligne avec la valeur donnée en parametre
function replaceMissing(line,value)
    n = length(line)
    for i=1:n
        if (ismissing(line[i]))
            line[i] = value
        end
    end
end

## Fonctions pour calculer les coefficients d'une regression ridge
Il faudra aussi retourner la moyenne et la variance pour déstandardizer les valeures

In [ ]:
# Fonction pour faire une regression ridge
# Ressort le beta, m, et s
function ridge(datas::DataFrame, station::Symbol)
       
    Train, Test = splitdataframe(datas, .75);
    # Prétraitement des données
    # Les variables avec les tildes correspondent à l'échantillon de test

    X = convert(Matrix{Int64},Train[:,Not(station)])
    m = mean(X, dims=1)
    s = std(X, dims=1)
    m[2] = 0
    s[2] = 1
    X = (X .- m) ./ s

    X̃ = convert(Matrix{Int64},Test[:,Not(station)])
    X̃ = (X̃ .- m) ./ s

    y = convert(Vector{Int64}, Train[:,station])
    m = mean(y)
    s = std(y)
    y = (y .- m) ./s

    ỹ = convert(Vector{Int64}, Test[:,station])
    ỹ = (ỹ .- m) ./s;

    #On calcule ensuite le RMSE pour chacun des valeurs de lambda
    RMSEs = DataFrame(λ=Float64[], RMSE=Float64[])

    for λ in 0:1:10000
   
        β̂ = (X'X + λ*I)\X'y
    
        ŷ = X̃*β̂
    
        ẽ = ỹ - ŷ
    
        RMSE = sqrt(dot(ẽ,ẽ)/length(ẽ))
    
        push!(RMSEs, [λ, RMSE])
    
    end
    
    # On trouve ensuite la valeure de lambda qui minimise le RMSE
    _, ind = findmin(RMSEs[:,:RMSE])

    λ̂ = RMSEs[ind,:λ]
    
    β̂ = (X'X + λ̂*I)\X'y
    
    #TODO validate model and print value of validator R² ajuste
    
    #On peut alors calculer les y avec les betas trouver et l'echantillon de test
    ŷ = X̃ * β̂
    ŷ = round.((ŷ .* s) .+ m)
    
    # Calcul du R² ajusté

    p = 4          # nombre de variables explicatives
    n = length(ỹ)  # taille de l'échantillon

    ỹ = (ỹ .* s) .+ m
    ȳ = mean(ỹ)
    e = ỹ - ŷ

    SST = sum( (ỹ[i] - ȳ)^2 for i=1:n )  # variabilité totale
    SSE = sum( e.^2 )                    # variabilité résiduelle

    R2aj =  1 - SSE/SST * (n-1)/(n-p)
    
    println("Le R² ajuste du modele trouve pour la station de $(station) est $(R2aj)")
    
    return β̂
end

In [ ]:
full_data = dropmissing(data, disallowmissing=true)
full_data = full_data[:,Not(:date)][:, Not(:heure)]
size(full_data)

In [ ]:
betas = DataFrame(station = Symbol[], β = Array{Float64}[])
for name in names(full_data)
    β̂ = ridge(full_data, name)
    push!(betas, [name, β̂])
end
betas

### Faire le remplissage des valeurs manquantes

In [ ]:
n = length(data[:,:date])
prec = data[:,Not(:date)][:,Not(:heure)]
nbMissing = 0

for row in eachrow(prec)
    nbMissing, ind = countMissing(row)
    if(nbMissing == 1)
        row[ind] = round((convert(Vector{Float64},row[Not(ind)])'*betas[:, :β][ind]))
    end
    # remplacer les lignes qui ont de 2 a 4 missing
    if(nbMissing<5 && nbMissing>1)
        replaceMissing(row,round(meanLine(row)))
    end
end
first(prec,10)

In [ ]:
prec.heure = data[:,:heure]
prec.date = data[:,:date]
first(prec,5)

In [ ]:
clean_values = prec[: ,[:date, :heure, :McTavish, :Bellevue, :Assomption, :Trudeau, :StHubert]]
CSV.write("clean_values.csv",clean_values)

# Analyse exploratoire

Cette section consitue une analyse exploratoire superficielle permettant de voir s'il existe un lien entre les précipitations et les surverses.

Prenons arbitrairement l'ouvrage de débordement près du Bota-Bota (4350-01D). La station météorologique la plus proche est McTavish. Prenons deux variables explicatives simple :
- la somme journalière des précipitations
- le taux horaire maximum journalier de précipitations

#### Calcul de la quantité journalière de précipitations pour chacune des stations météorologiques

In [ ]:
pcp_sum = by(prec, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)
first(pcp_sum ,10)

#### Extraction du taux horaire journalier maximum des précipitations pour chacune des stations météorologiques

In [ ]:
pcp_max = by(prec, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(pcp_max,10)

#### Inclusion dans un dataframe de ces deux variables explicatives potentielles

In [ ]:
ouvrage = "4350-01D"

df = filter(row -> row.NO_OUVRAGE == ouvrage, surverse_df)

x₁ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var somme journalière--McTavish
x₂ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var max journalier-----McTavish
x₃ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var somme journalière--Bellevue
x₄ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var max journalier-----Bellevue
x₅ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var somme journalière--
x₆ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var max journalier-----
x₇ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var somme journalière
x₈ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var max journalier
x₉ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var somme journalière
x₁₀ = Array{Union{Missing, Int64}}(missing, size(df,1)) # var max journalier



for i=1:size(df,1)
    
    ind = findfirst(pcp_sum[:,:date] .== df[i,:DATE])
    x₁[i] = pcp_sum[ind,:McTavish]
    ind = findfirst(pcp_max[:,:date] .== df[i,:DATE])
    x₂[i] = pcp_max[ind,:McTavish]
    
    ind = findfirst(pcp_sum[:,:date] .== df[i,:DATE])
    x₃[i] = pcp_sum[ind,:Bellevue]
    ind = findfirst(pcp_max[:,:date] .== df[i,:DATE])
    x₄[i] = pcp_max[ind,:Bellevue]
    
    ind = findfirst(pcp_sum[:,:date] .== df[i,:DATE])
    x₅[i] = pcp_sum[ind,:Assomption]
    ind = findfirst(pcp_max[:,:date] .== df[i,:DATE])
    x₆[i] = pcp_max[ind,:Assomption]
    
    ind = findfirst(pcp_sum[:,:date] .== df[i,:DATE])
    x₇[i] = pcp_sum[ind,:Trudeau]
    ind = findfirst(pcp_max[:,:date] .== df[i,:DATE])
    x₈[i] = pcp_max[ind,:Trudeau]
    
    ind = findfirst(pcp_sum[:,:date] .== df[i,:DATE])
    x₉[i] = pcp_sum[ind,:StHubert]
    ind = findfirst(pcp_max[:,:date] .== df[i,:DATE])
    x₁₀[i] = pcp_max[ind,:StHubert]
    
end

df[!,:sumMcTavish] = x₁
df[!,:maxMcTavish] = x₂
df[!,:sumBellevue] = x₃
df[!,:maxBellevue] = x₄
df[!,:sumAssomption] = x₅
df[!,:maxAssomption] = x₆
df[!,:sumTrudeau] = x₇
df[!,:maxTrudeau] = x₈
df[!,:sumStHubert] = x₉
df[!,:maxStHubert] = x₁₀


#dropmissing!(df, [:SUM, :MAX],disallowmissing=true)
first(df,10)

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que la somme des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [ ]:
#plot(df, x=:SURVERSE, y=:SUM, Geom.boxplot)

#### Traçage des distribution de la somme des précipitations en fonction des surverses ou non

On remarque que les deux distributions sont très différentes. Ceci suggère que le maximum journalier des précipitations à la station McTavish a un effet sur les surverses au Bota-Bota.

In [ ]:
#plot(df, x=:SURVERSE, y=:MAX, Geom.boxplot)

# Création du fichier de prédictions pour soumettre sur Kaggle

Dans ce cas-ci, nous prédirons une surverse avec une probabilité de 1/2 sans considérer aucune variable explicative.

In [ ]:

# Chargement du fichier de test
test = CSV.read("data/test.csv")

# Pour chacune des lignes du fichier test, comportant un ouvrage et une date, une prédiction est requise.
# Dans ce cas-ci, utilisons une prédiction les plus naîve. 
# On prédit avec une chance sur deux qu'il y ait surverse, sans utiliser de variables explicatives
n = size(test,1)
surverse = rand(n) .> .5


# Création du fichier sampleSubmission.csv pour soumettre sur Kaggle
ID = test[:,:NO_OUVRAGE].*"_".*string.(test[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=surverse)
CSV.write("sampleSubmission.csv",sampleSubmission)

# Vous pouvez par la suite déposer le fichier sampleSubmission.csv sur Kaggle.
